<a href="https://colab.research.google.com/github/PrinceWallyy/MachineLearning/blob/master/ACML_Assignment_2_Exercise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Weights & Biases

In [ ]:
!pip install wandb -q

     |████████████████████████████████| 1.7 MB 7.2 MB/s 
     |████████████████████████████████| 97 kB 8.1 MB/s 
     |████████████████████████████████| 139 kB 42.6 MB/s 
     |████████████████████████████████| 180 kB 55.5 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 


In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Preprocess

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.datasets import cifar10
from keras.utils import np_utils

np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
(x_train, _), (x_test, _) = cifar10.load_data()
data = np.vstack((x_train, x_test))
data = data / 255
(x_train, x_val, x_test) = np.vsplit(data, np.array([48000, 54000]))
assert x_train.shape == (48000, 32, 32, 3)
assert x_val.shape == (6000, 32, 32, 3)
assert x_test.shape == (6000, 32, 32, 3)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

170508288/170498071 [==============================] - 4s 0us/step


# Sweep

In [ ]:
sweep_config = {
  "name" : "CAE_Random_Optimization",
  "method" : "random",
  "metric":{
    "name": "val_loss",
    "goal": "minimize"
  },
  "parameters" : {
    "kernel" :{
      "values" :[1, 3, 5, 7]
    },
    "stride" :{
      "values" :[1]
    },
    "padding" :{
      "values" :['same']
    },
    "layer_1_filter" :{
      "values" :[8, 12, 16, 32]
    },
    "layer_2_filter" :{
      "values" :[12, 16, 32, 64]
    },
    "layer_3_filter" :{
      "values" :[16, 32, 64, 128]
    },
    "layer_4_filter" :{
      "values" :[12, 16, 32, 64]
    }
  }
}

sweep_id = wandb.sweep(sweep_config, project="Assignment 2")

Create sweep with ID: 996y9ysq
Sweep URL: https://wandb.ai/princewallyy/Assignment%202/sweeps/996y9ysq


# Convolutional AutoEncoder

In [ ]:
input_shape = (32, 32, 3)

In [ ]:
def createModel(config):
  model = models.Sequential()
  model.add(layers.Conv2D(filters=config.layer_1_filter, kernel_size=(config.kernel, config.kernel), strides=config.stride,
                          padding=config.padding, activation='relu', input_shape=input_shape))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))

  model.add(layers.Conv2D(filters=config.layer_2_filter, kernel_size=(config.kernel, config.kernel), strides=config.stride,
                          padding=config.padding, activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))

  model.add(layers.Conv2D(filters=config.layer_3_filter, kernel_size=(config.kernel, config.kernel), strides=config.stride,
                          padding=config.padding, activation='relu'))
  model.add(layers.UpSampling2D(size=(2, 2)))

  model.add(layers.Conv2D(filters=config.layer_4_filter, kernel_size=(config.kernel, config.kernel), strides=config.stride,
                          padding=config.padding, activation='relu'))
  model.add(layers.UpSampling2D(size=(2, 2)))

  model.add(layers.Conv2D(3, (3, 3), padding='same', activation='sigmoid'))

  return model

# Training

In [ ]:
def train():
  with wandb.init() as run:
    config = wandb.config
    model = createModel(config)

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

    model.fit(x_train, x_train, epochs=10, validation_data=(x_val, x_val), callbacks=[WandbCallback()])

    loss, accuracy = model.evaluate(x_test, x_test)
    wandb.log({'Test Error Rate': round((1 - accuracy) * 100, 2)})

In [ ]:
wandb.agent(sweep_id, function=train, count=20, project="Assignment 2")